In [17]:
import pandas as pd
import numpy as np
import os
import h3 
import re

In [20]:
poi_df = pd.read_csv('mapped_data.csv')

# 2. 計算 H3 索引 (使用 list comprehension 加速)
# 注意：確保你的 CSV 裡確實有 'latitude' 和 'longitude' 這兩個欄位
# 如果你的欄位是 'wgs84_纬度'，請將下方 lat, lon 改成對應欄位
try:
    poi_df['h3_l7'] = [
        h3.geo_to_h3(lat, lon, 7) 
        for lat, lon in zip(poi_df['latitude'], poi_df['longitude'])
    ]
except KeyError:
    print("錯誤：找不到 'latitude' 或 'longitude' 欄位，請檢查 CSV 欄位名稱")

# 3. 處理欄位刪除
# 修正了列表的語法 (加上引號和逗號)
columns_to_drop = [
    'id', 'pname', 'pcode', 'cityname', 'citycode', 'adname', 'adcode', 
    'address', 'location', 'tel', 'business_area', '行业大类', 
    'typecode', 'timestamp', 'wgs84_经度', 'wgs84_纬度' ,'行业中类'
]

# 使用 errors='ignore' 防止如果欄位不存在時報錯
poi_df = poi_df.drop(columns=columns_to_drop, errors='ignore')

# 4. 處理 amenity/osm_tag
# 我們之前生成的欄位叫 'osm_tag'。
# 如果你想把它改名為 'amenity' 以符合你的後續需求：
if 'osm_tag' in poi_df.columns:
    poi_df = poi_df.rename(columns={'osm_tag': 'amenity'})

# 5. 過濾邏輯 (請確認你的需求)
# 原本寫法: poi_df[poi_df.amenity.isna()] 會保留 amenity 是空的資料 (通常是不要的)
# 建議寫法: 保留 amenity "不是" 空的資料
if 'amenity' in poi_df.columns:
    poi_df = poi_df[poi_df['amenity'].notna()]

# 顯示結果
print(poi_df.head())

          name         行业小类   longitude   latitude     amenity  \
0         志方人家   中餐厅|体育休闲服务  119.724353  31.183822  restaurant   
1       欣竹山庄饭店   中餐厅|体育休闲服务  119.724059  31.184167  restaurant   
2       阿兴农家小院   中餐厅|体育休闲服务  119.800123  31.317220  restaurant   
3         上善酒家  综合酒楼|体育休闲服务  119.675089  31.297060  restaurant   
4  小龙潭农家乐(龙山店)   中餐厅|体育休闲服务  119.757987  31.192809  restaurant   

             h3_l7  
0  87309e0f4ffffff  
1  87309e0f4ffffff  
2  87309e066ffffff  
3  87309e023ffffff  
4  87309e0e2ffffff  


In [21]:
import pandas as pd
import numpy as np

target_col = 'amenity' 

# 1. 快速計算每個 H3 網格內的各類別數量
# index=列(H3 ID), columns=欄(設施類型), values=計數
counts_df = pd.crosstab(poi_df['h3_l7'], poi_df[target_col])

# 2. 準備所有的 H3 ID (包含 poi_df 和 ohca_df 的聯集)
# 確保即使某個 H3 格子裡沒有 POI，但在 OHCA 資料裡有，也能保留下來
all_h3_ids = np.unique(np.concatenate((
    poi_df['h3_l7'].unique(), 
)))

# 3. 建立最終 DataFrame 並合併數據
h3_l7_df = pd.DataFrame({'id': all_h3_ids})

# 將計算好的 counts_df 合併進來
# left_on='id' 對應 h3_l7_df 的 id
# right_index=True 對應 counts_df 的 index (也就是 h3_l7)
h3_l7_df = h3_l7_df.merge(counts_df, left_on='id', right_index=True, how='left')

# 4. 填充缺失值 (NaN 補 0)
# 因為有些 H3 格子在 ohca_df 裡有，但在 poi_df 裡完全沒資料，合併後會變成 NaN
h3_l7_df = h3_l7_df.fillna(0)

# 5. (可選) 顯示統計資訊
print(f"總共處理了 {len(h3_l7_df)} 個唯一的 H3 網格")
print(f"總共生成了 {counts_df.shape[1]} 個特徵欄位 (amenity types)")
print("欄位列表:", counts_df.columns.tolist())

# 預覽結果
h3_l7_df

總共處理了 432 個唯一的 H3 網格
總共生成了 52 個特徵欄位 (amenity types)
欄位列表: ['apartments', 'arts_centre', 'atm', 'bank', 'bar', 'barn', 'bus_station', 'cafe', 'car_wash', 'charging_station', 'cinema', 'clinic', 'college', 'commercial', 'dentist', 'detached', 'dojo', 'dormitory', 'driving_school', 'exhibition_centre', 'fast_food', 'fuel', 'gambling', 'grave_yard', 'greenhouse', 'hospital', 'hotel', 'ice_cream', 'industrial', 'internet_cafe', 'kindergarten', 'library', 'loading_dock', 'marketplace', 'nightclub', 'office', 'parking', 'parking_entrance', 'pharmacy', 'post_office', 'public', 'residential', 'restaurant', 'retail', 'school', 'service', 'sports_centre', 'theatre', 'training', 'university', 'veterinary', 'warehouse']


,id,apartments,arts_centre,atm,bank,bar,barn,bus_station,cafe,car_wash,...,restaurant,retail,school,service,sports_centre,theatre,training,university,veterinary,warehouse
0,87309e000ffffff,0,0,0,0,0,0,3,0,0,...,0,1,0,1,0,0,0,0,0,0
1,87309e001ffffff,0,0,0,0,0,0,9,0,0,...,0,3,0,0,1,0,0,0,0,0
2,87309e002ffffff,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,87309e003ffffff,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,87309e004ffffff,0,0,1,3,0,0,7,1,1,...,34,55,1,18,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,87309ebb2ffffff,9,0,1,1,1,0,10,4,0,...,57,58,2,45,7,0,8,0,0,8
428,87309ebb3ffffff,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
429,87309ebb4ffffff,0,0,0,0,0,0,5,0,0,...,0,1,0,1,0,0,0,0,0,1
430,87309ebb5ffffff,0,0,1,4,0,0,6,0,0,...,36,45,2,36,0,0,0,0,0,3


In [28]:
import pandas as pd

df_new_structure = pd.read_csv('h3_l7_df_new.csv')
cols_to_remove = ['ohca', 'commercial;yes']
new_columns = df_new_structure.columns[~df_new_structure.columns.isin(cols_to_remove)]

h3_l7_df_yixing = h3_l7_df.copy()

# 3. 補齊缺少的欄位
for col in new_columns:
    if col not in h3_l7_df_yixing.columns:
        h3_l7_df_yixing[col] = 0

print("h3_l7_df_yixing 形狀:", h3_l7_df_yixing.shape)
print(h3_l7_df_yixing.head())

h3_l7_df_yixing 形狀: (432, 115)
                id  apartments  arts_centre  atm  bank  bar  barn  \
0  87309e000ffffff           0            0    0     0    0     0   
1  87309e001ffffff           0            0    0     0    0     0   
2  87309e002ffffff           0            0    0     0    0     0   
3  87309e003ffffff           0            0    0     0    0     0   
4  87309e004ffffff           0            0    1     3    0     0   

   bus_station  cafe  car_wash  ...  church  terrace  pavilion  stadium  \
0            3     0         0  ...       0        0         0        0   
1            9     0         0  ...       0        0         0        0   
2            4     0         0  ...       0        0         0        0   
3            0     0         0  ...       0        0         0        0   
4            7     1         1  ...       0        0         0        0   

   cabin  boathouse  riding_hall  construction  ship  ruins  
0      0          0            0         

In [29]:
h3_l7_df_yixing.to_csv("h3_l7_df_yixing.csv", index=False)


In [24]:
# 假設 h3_l7_df 和 df_new_structure 已經被讀取為 DataFrame

# 1. 取得兩個 DataFrame 的欄位名稱集合
cols_original = set(h3_l7_df.columns)
cols_new = set(df_new_structure.columns)

# 2. 找出重複的欄位 (交集)
# 這些是兩個 DataFrame 都有的欄位
intersection_cols = cols_original.intersection(cols_new)

# 3. 找出不重複的欄位 (差集)
# 只在原本 h3_l7_df 裡有的欄位 (新表沒有的)
only_in_original = cols_original - cols_new

# 只在新表 df_new_structure 裡有的欄位 (原本沒有的)
only_in_new = cols_new - cols_original

# 4. 印出結果
print(f"=== 欄位比較結果 ===")
print(f"兩邊都有的重複欄位 (共 {len(intersection_cols)} 個):")
print(sorted(list(intersection_cols))) # 排序後印出比較好讀
print("-" * 30)

print(f"只在 h3_l7_df (舊表) 的欄位 (共 {len(only_in_original)} 個):")
print(sorted(list(only_in_original)))
print("-" * 30)

print(f"只在 df_new_structure (新表) 的欄位 (共 {len(only_in_new)} 個):")
print(sorted(list(only_in_new)))

=== 欄位比較結果 ===
兩邊都有的重複欄位 (共 53 個):
['apartments', 'arts_centre', 'atm', 'bank', 'bar', 'barn', 'bus_station', 'cafe', 'car_wash', 'charging_station', 'cinema', 'clinic', 'college', 'commercial', 'dentist', 'detached', 'dojo', 'dormitory', 'driving_school', 'exhibition_centre', 'fast_food', 'fuel', 'gambling', 'grave_yard', 'greenhouse', 'hospital', 'hotel', 'ice_cream', 'id', 'industrial', 'internet_cafe', 'kindergarten', 'library', 'loading_dock', 'marketplace', 'nightclub', 'office', 'parking', 'parking_entrance', 'pharmacy', 'post_office', 'public', 'residential', 'restaurant', 'retail', 'school', 'service', 'sports_centre', 'theatre', 'training', 'university', 'veterinary', 'warehouse']
------------------------------
只在 h3_l7_df (舊表) 的欄位 (共 0 個):
[]
------------------------------
只在 df_new_structure (新表) 的欄位 (共 64 個):
['animal_boarding', 'animal_shelter', 'bbq', 'bench', 'bicycle_parking', 'bicycle_rental', 'bicycle_repair_station', 'boathouse', 'bunker', 'cabin', 'car_rental', 'ch